#### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-credentials.json", scope
)
gc = gspread.authorize(credentials)

In [7]:
spreadsheet_key = "1_RXzXkHPEyDAiDKmz98wTSC9UNo-8OyjGTT540X_vJk"
book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

#### get chinese real estate data from FRED

In [9]:
# chinese real estaee series=QCNR628BIS
import urllib.request, json

with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=QCNR628BIS&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())
    print(data["observations"])

[{'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2005-04-01', 'value': '87.9389'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2005-07-01', 'value': '89.5385'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2005-10-01', 'value': '89.7531'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2006-01-01', 'value': '88.5259'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2006-04-01', 'value': '90.6989'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2006-07-01', 'value': '92.4696'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2006-10-01', 'value': '92.4653'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2007-01-01', 'value': '90.7212'}, {'realtime_start': '2022-08-25', 'realtime_end': '2022-08-25', 'date': '2007-04-01', 'value': '92.6815'}, {'realtime_start': '2022-08-25', 'realtime_en

In [10]:
frames = []
for point in data["observations"]:
    frames.append(pd.DataFrame(point, index=[0]))

In [11]:
df = pd.concat(frames).reset_index(drop=True)

In [12]:
dw.add_data(chart_id="ZnOCh", data=df[["date", "value"]])

<Response [204]>

from source

In [13]:
df_yoy = pd.read_excel(
    "https://www.bis.org/statistics/pp/pp_selected.xlsx",
    sheet_name="Quarterly Series",
    skiprows=3,
)[["Period", "Q:CN:R:771"]]

In [14]:
df_yoy.dtypes

Period        datetime64[ns]
Q:CN:R:771           float64
dtype: object

In [15]:
dw.add_data(
    chart_id="rFdW6",
    data=(df_yoy[(~df_yoy["Q:CN:R:771"].isna()) & (df_yoy["Period"] >= "2010-01-01")]),
)

<Response [204]>

#### real estate github

In [54]:
df = pd.read_json(
    "https://raw.githubusercontent.com/WeNeedHome/SummaryOfLoanSuspension/main/data/generated/cities-for-visualization.json"
)

In [55]:
df[["longitude", "latitude"]] = pd.json_normalize(df["pos"])

In [ ]:
#dw.add_data(chart_id="a3ySL", data=df)

<Response [204]>

summary info 

In [57]:
df["city"].nunique()

98

In [60]:
df["count"].sum()

328

In [65]:
city_ck = ["长沙市", "衡阳市", "郑州市", "成都市"]

In [67]:
df[df["city"].isin(city_ck)]

,province,city,pos,count,district,longitude,latitude
43,河南省,郑州市,"{'lng': 113.38324, 'lat': 34.786948}",1,荥阳市,113.383240,34.786948
45,河南省,郑州市,"{'lng': 113.625328, 'lat': 34.746611}",44,NaN,113.625328,34.746611
57,湖南省,衡阳市,"{'lng': 112.953168, 'lat': 27.08117}",1,衡东县,112.953168,27.081170
58,湖南省,衡阳市,"{'lng': 112.572018, 'lat': 26.893368}",1,NaN,112.572018,26.893368
60,湖南省,长沙市,"{'lng': 113.643076, 'lat': 28.162833}",1,浏阳市,113.643076,28.162833
65,湖南省,长沙市,"{'lng': 112.938884, 'lat': 28.22808}",11,NaN,112.938884,28.228080
100,四川省,成都市,"{'lng': 104.066794, 'lat': 30.572893}",9,NaN,104.066794,30.572893
102,四川省,成都市,"{'lng': 103.647153, 'lat': 30.988767}",1,都江堰市,103.647153,30.988767


In [20]:
df.to_csv("data/processed/realestate.csv")

In [22]:
cities = (
    df.groupby("city").agg(total=pd.NamedAgg(column="count", aggfunc=sum)).reset_index()
)

In [23]:
# You can also do it like this:

# cities = (
#     df.groupby(["city"]).agg({'count': sum}).reset_index()
# )

In [51]:
cities.sort_values("total", ascending=False).reset_index()

In [53]:
cities["city"].nunique()

98

In [25]:
cities["cityname"] = cities["city"].str.strip("市")

In [26]:
city_details = pd.read_table(
    "https://raw.githubusercontent.com/public-wheels/china-cities/master/china_cities.txt"
).rename(
    columns={
        "英文": "cityname_eng",
        "中文": "cityname",
        "纬度": "latitude",
        "经度": "longitude",
    }
)

In [27]:
# This is why:
city_details[city_details["cityname_eng"] == "zhongshan"]

,城市/地区编码,cityname_eng,cityname,国家代码,国家英文,国家中文,省英文,省中文,所属上级市英文,所属上级市中文,latitude,longitude
322,CN101070208,zhongshan,中山,CN,China,中国,liaoning,辽宁,dalian,大连,38.921553,121.643760
2513,CN101260805,zhongshan,钟山,CN,China,中国,guizhou,贵州,liupanshui,六盘水,26.584805,104.846244
2842,CN101281701,zhongshan,中山,CN,China,中国,guangdong,广东,zhongshan,中山,22.521113,113.382391
3073,CN101300704,zhongshan,钟山,CN,China,中国,guangxi,广西,hezhou,贺州,24.528566,111.303629


In [28]:
city_details[city_details["cityname_eng"] == "xian"]

,城市/地区编码,cityname_eng,cityname,国家代码,国家英文,国家中文,省英文,省中文,所属上级市英文,所属上级市中文,latitude,longitude
137,CN101050311,xian,西安,CN,China,中国,heilongjiang,黑龙江,mudanjiang,牡丹江,44.581032,129.613110
286,CN101060705,xian,西安,CN,China,中国,jilin,吉林,liaoyuan,辽源,42.920415,125.151424
829,CN101110101,xian,西安,CN,China,中国,shan-xi,陕西,xian,西安,34.263161,108.948024


In [29]:
final = cities.merge(city_details, on="cityname").drop(
    columns=["城市/地区编码", "国家代码", "国家英文", "省英文", "省中文", "所属上级市英文", "所属上级市中文", "国家中文"]
)

In [30]:
final["cityname_eng"] = final["cityname_eng"].str.title()

In [31]:
dw.add_data(chart_id="HTJiI", data=final)

<Response [204]>

why are there duplicates for zhongshan and xian?

In [32]:
final.dtypes

city             object
total             int64
cityname         object
cityname_eng     object
latitude        float64
longitude       float64
dtype: object

nbs data

In [33]:
# Opening the worksheet by using Worksheet ID
workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
values = sheet.get_all_values()
nbs = pd.DataFrame(values[1:], columns=values[0]).transpose()

In [34]:
nbs.columns = nbs.iloc[0]
df = nbs[1:].reset_index().rename(columns={"index": "month"})

In [35]:
df["date"] = pd.to_datetime(df["month"], format="%b %Y")

In [36]:
df.sort_values("date", ascending=True)

Indicators,month,started,completed,sold,date
90,Jan 2015,0,0,0,2015-01-01
89,Feb 2015,9851.53,7804.44,4982.92,2015-02-01
88,Mar 2015,16791.32,12392.49,10062.17,2015-03-01
87,Apr 2015,25081.11,15526.96,14915.75,2015-04-01
86,May 2015,35090.6,19598.76,20731.52,2015-05-01
85,Jun 2015,46890.84,24354.42,28941.12,2015-06-01
84,Jul 2015,56684.05,28077.44,34884.45,2015-07-01
83,Aug 2015,65829.51,31494.47,40724.07,2015-08-01
82,Sep 2015,79345,37791.74,47898,2015-09-01
81,Oct 2015,87752.7,45235.05,54702.71,2015-10-01


In [37]:
df = df[["date", "sold", "started", "completed"]]

In [38]:
df["sold"] = df["sold"].astype(float) / 100
df["started"] = df["started"].astype(float) / 100
df["completed"] = df["completed"].astype(float) / 100

In [ ]:
dw.add_data(chart_id="ljQcp", data=df)

<Response [204]>

In [40]:
sold = df[["date", "sold"]].sort_values("date", ascending=True)
sold["type"] = "sold"
sold["monthly"] = sold["sold"] - sold["sold"].shift(periods=1)

In [42]:
started = df[["date", "started"]].rename(columns={"started": "value"})
started["type"] = "started"

In [43]:
completed = df[["date", "completed"]].rename(columns={"completed": "value"})
completed["type"] = "completed"

In [44]:
df_long = pd.concat([sold, started, completed])

In [68]:
alt.Chart(df_long).mark_area().encode(
    x="date", y="rollmean", color="type", row="type"
).properties(height=100)

ValueError: rollmean encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

In [46]:
alt.Chart(df_long).mark_line(point=True).encode(
    x=alt.X("date"), y=alt.Y("value"), color="type"
)

alt.Chart(...)

In [47]:
grouped_col["y"] = 2

NameError: name 'grouped_col' is not defined

In [ ]:
base_chart = (
    alt.Chart(grouped_col)
    .mark_bar()
    .encode(
        x=alt.X("date", stack="normalize"),
        y=alt.Y("y"),
        color=alt.Color(
            ("value"),
            scale=alt.Scale(scheme="teals", reverse=False),
        ),
    )
    .properties(width=35, height=100)
)

alt.layer(base_chart, data=grouped_col).facet("type")

In [ ]:
base_chart = (
    alt.Chart(df_long)
    .mark_bar()
    .encode(
        x=alt.X("date", stack="normalize"),
        y=alt.Y("y"),
        color=alt.Color(
            ("value"),
            scale=alt.Scale(scheme="teals", reverse=False),
        ),
    )
    .properties(width=75, height=100)
)

alt.layer(base_chart, data=df_long).facet("type")